In [1]:
import os
import json
import pickle
import numpy as np
from pathlib import Path
from typing import Dict
from tqdm.auto import tqdm  # Import tqdm for the progress bar

MATHLIB_DIR = Path('~/Mathlib') # Mathlib dir
LeanDojo_DIR = Path('~/leandojo_benchmark_4/') # LeanDojo dir
Save_DIR = '~/STP/SFT/' # save dir

os.makedirs(Save_DIR, exist_ok=True)

# Read Lean Files

In [2]:
def read_lean_files(directory: Path) -> Dict[str, str]:
    """
    Recursively reads all `.lean` files in the specified directory and returns
    a dictionary mapping file paths to their contents, displaying a progress bar.

    Args:
        directory (Path): The root directory to start searching for `.lean` files.

    Returns:
        Dict[str, str]: A dictionary where keys are file paths (as strings) and
                        values are the contents of the files.
    """
    if not directory.exists():
        raise FileNotFoundError(f"The directory {directory} does not exist.")
    if not directory.is_dir():
        raise NotADirectoryError(f"The path {directory} is not a directory.")

    # Collect all .lean files first to determine the total number for the progress bar
    lean_files = list(directory.rglob('*.lean'))
    total_files = len(lean_files)
    print(f"Found {total_files} `.lean` files to process.")

    file_contents = {}

    # Initialize tqdm progress bar
    with tqdm(total=total_files, desc="Reading .lean files", unit="file") as pbar:
        for file_path in lean_files:
            try:
                # Read the file content with UTF-8 encoding
                with file_path.open() as f:
                    content = f.readlines()
                # Use the absolute path as the key
                file_contents[str(file_path.resolve())] = content
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
            finally:
                pbar.update(1)  # Update the progress bar

    return file_contents


try:
    lean_file_dict = read_lean_files(MATHLIB_DIR)
    lean_file_dict = {k.split('LeanDojo/mathlib4/')[-1]: v for k, v in lean_file_dict.items()}
    print(f"Total .lean files read: {len(lean_file_dict)}")
    # Example: Print the first file's path and its first 100 characters
    if lean_file_dict:
        first_path = next(iter(lean_file_dict))
        print(f"First file: {first_path}")
        print(f"Content snippet: {lean_file_dict[first_path][:10]}")
except Exception as error:
    print(f"An error occurred: {error}")

Found 4360 `.lean` files to process.


Reading .lean files:   0%|          | 0/4360 [00:00<?, ?file/s]

Total .lean files read: 4360
First file: Mathlib/Tactic.lean
Content snippet: ['import Mathlib.Tactic.Abel\n', 'import Mathlib.Tactic.AdaptationNote\n', 'import Mathlib.Tactic.ApplyAt\n', 'import Mathlib.Tactic.ApplyCongr\n', 'import Mathlib.Tactic.ApplyFun\n', 'import Mathlib.Tactic.ApplyWith\n', 'import Mathlib.Tactic.ArithMult\n', 'import Mathlib.Tactic.ArithMult.Init\n', 'import Mathlib.Tactic.Attr.Core\n', 'import Mathlib.Tactic.Attr.Register\n']


# Read LeanDojo Data

In [3]:
train_path = LeanDojo_DIR / "random/train.json"
val_path = LeanDojo_DIR / "random/val.json"
test_path = LeanDojo_DIR / "random/test.json"
proofs_train = json.load(train_path.open())
proofs_val = json.load(val_path.open())
proofs_test = json.load(test_path.open())

In [4]:
print(len(proofs_train))
print(len(proofs_val))
print(len(proofs_test))

117068
2000
2000


In [5]:
from collections import defaultdict

entry_dict = defaultdict(list)
for entry in proofs_train + proofs_val + proofs_test:
    if len(entry['traced_tactics']) > 0:
        entry_dict[entry['file_path']].append(entry)
print(len(entry_dict))
print(sum(len(v) for v in entry_dict.values()))

3538
60751


In [6]:
def get_statement(theorem_str):
    return theorem_str.split(':=')[0].strip()

In [7]:
mathlib_theorems = []
theorem_dict = {}

empty_file = 0
for file_name in tqdm(lean_file_dict.keys()):
    if (len(entry_dict[file_name]) == 0):
        empty_file += 1
        continue
    lines = lean_file_dict[file_name]
    entries = entry_dict[file_name]
    entries = sorted(entries, key = lambda x: x['start'][0])
    
    for entry in entries:
        theorem_str = ''.join(lines[entry['start'][0] - 1: entry['end'][0]])
        theorem_item = (file_name, theorem_str)
        mathlib_theorems.append(theorem_item)
        theorem_dict[entry['full_name']] = theorem_item
print('[warning] # Files without theorem:', empty_file)
print('# Mathlib theorems:', len(mathlib_theorems))

  0%|          | 0/4360 [00:00<?, ?it/s]

[warning] # Files without theorem: 864
# Mathlib theorems: 59744


In [8]:
sum_premises = 0
conjecture_examples = []

try:
    for file_name in tqdm(lean_file_dict.keys()):
        if (len(entry_dict[file_name]) == 0):
            empty_file += 1
            continue
    
        theorems_in_file = []
        lines = lean_file_dict[file_name]
        entries = entry_dict[file_name]
        entries = sorted(entries, key = lambda x: x['start'][0])
        
        for entry in entries:
            theorem_str = ''.join(lines[entry['start'][0] - 1: entry['end'][0]])
            contexts = ''.join(lines[:entry['start'][0] - 1]).strip()
            
            premises = set()
            for tactics in entry['traced_tactics']:
                for p in tactics['annotated_tactic'][1]:
                    if p['full_name'] in theorem_dict:
                        premises.add(p['full_name'])
    
            for p_name in premises:
                for p_context, p_theorem, p_premises in reversed(theorems_in_file):
                    if p_name in p_premises:
                        conjecture_examples.append((file_name, p_context, p_theorem, theorem_dict[p_name][1], theorem_str))
            
            theorems_in_file.append((contexts, theorem_str, premises))
            sum_premises += len(premises)
except:
    pass
print(sum_premises)
print('# Conjecture examples:', len(conjecture_examples))

  0%|          | 0/4360 [00:00<?, ?it/s]

79908
# Conjecture examples: 51273


In [9]:
conjecture_examples[-1]

('Mathlib/Geometry/Manifold/MFDeriv/UniqueDifferential.lean',
 "/-\nCopyright (c) 2020 Sébastien Gouëzel. All rights reserved.\nReleased under Apache 2.0 license as described in the file LICENSE.\nAuthors: Sébastien Gouëzel, Floris van Doorn\n-/\nimport Mathlib.Geometry.Manifold.MFDeriv.Atlas\n\n/-!\n# Unique derivative sets in manifolds\n\nIn this file, we prove various properties of unique derivative sets in manifolds.\n* `image_denseRange`: suppose `f` is differentiable on `s` and its derivative at every point of `s`\nhas dense range. If `s` has the unique differential property, then so does `f '' s`.\n* `uniqueMDiffOn_preimage`: the unique differential property is preserved by local diffeomorphisms\n* `uniqueDiffOn_target_inter`: the unique differential property is preserved by\n  pullbacks of extended charts\n* `tangentBundle_proj_preimage`: if `s` has the unique differential property,\n  its preimage under the tangent bundle projection also has\n-/\n\nnoncomputable section\n\nope

# Construct dataset

In [10]:
START_LEMMA_STMT = '<easy theorem>'
START_THM = '<hard theorem>'
END_THM = '</hard theorem>'
INVOKED_LEMMA = '<lemma>'

def format_conjecture_example(context, easy_theorem, shared_lemma, hard_theorem):
    prompt = f'Complete the following Lean 4 code:\n\n```lean4\n{context.strip()}\n' \
          f'{INVOKED_LEMMA}\n{shared_lemma.strip()}\n{START_LEMMA_STMT}\n' \
          f'{easy_theorem.strip()}\n{START_THM}'
    target = f'\n{get_statement(hard_theorem).strip()}\n{END_THM}'
    return {'prompt': prompt, 'target': target}

conjecture_ds = []
for file_name, context, theorem, shared_lemma, theorem_str in conjecture_examples:
    conjecture_ds.append(format_conjecture_example(context, theorem, shared_lemma, theorem_str))

print(len(conjecture_ds))

51273


In [11]:
with open(os.path.join(Save_DIR, 'conjecture.json'), 'w') as f:
    json.dump(conjecture_ds, f)

In [12]:
print(conjecture_ds[10]['prompt'])
print(conjecture_ds[10]['target'])

Complete the following Lean 4 code:

```lean4
/-
Copyright (c) 2021 Bryan Gin-ge Chen. All rights reserved.
Released under Apache 2.0 license as described in the file LICENSE.
Authors: Adam Topaz, Bryan Gin-ge Chen, Yaël Dillies
-/
import Mathlib.Order.BooleanAlgebra
import Mathlib.Logic.Equiv.Basic

#align_import order.symm_diff from "leanprover-community/mathlib"@"6eb334bd8f3433d5b08ba156b8ec3e6af47e1904"

/-!
# Symmetric difference and bi-implication

This file defines the symmetric difference and bi-implication operators in (co-)Heyting algebras.

## Examples

Some examples are
* The symmetric difference of two sets is the set of elements that are in either but not both.
* The symmetric difference on propositions is `Xor'`.
* The symmetric difference on `Bool` is `Bool.xor`.
* The equivalence of propositions. Two propositions are equivalent if they imply each other.
* The symmetric difference translates to addition when considering a Boolean algebra as a Boolean
  ring.

## Main de

In [13]:
with open(os.path.join(Save_DIR, 'theorem_dict.pkl'), 'wb') as f:
    pickle.dump(theorem_dict, f)

In [14]:
print(len(theorem_dict))

59742


In [15]:
theorem_dict['nat_sub_dvd_pow_sub_pow'] # example

('Mathlib/Algebra/GeomSum.lean',
 'theorem nat_sub_dvd_pow_sub_pow (x y n : ℕ) : x - y ∣ x ^ n - y ^ n := by\n  rcases le_or_lt y x with h | h\n  · have : y ^ n ≤ x ^ n := Nat.pow_le_pow_left h _\n    exact mod_cast sub_dvd_pow_sub_pow (x : ℤ) (↑y) n\n  · have : x ^ n ≤ y ^ n := Nat.pow_le_pow_left h.le _\n    exact (Nat.sub_eq_zero_of_le this).symm ▸ dvd_zero (x - y)\n')

## eval dataset

In [16]:
PROVER_PROMPT = 'Complete the following Lean 4 code:\n\n```lean4\n'
eval_theorems = []

empty_file = 0
for file_name in tqdm(lean_file_dict.keys()):
    if (len(entry_dict[file_name]) == 0):
        empty_file += 1
        continue
    lines = lean_file_dict[file_name]
    entries = entry_dict[file_name]
    entries = sorted(entries, key = lambda x: x['start'][0])
    
    for entry in entries:
        context_str = PROVER_PROMPT + ''.join(lines[:entry['start'][0] - 1])
        theorem_str = ''.join(lines[entry['start'][0] - 1: entry['end'][0]])
        theorem_item = (file_name, context_str, theorem_str)
        eval_theorems.append(theorem_item)
print('# Mathlib theorems:', len(eval_theorems))

  0%|          | 0/4360 [00:00<?, ?it/s]

# Mathlib theorems: 59744


In [17]:
rng = np.random.default_rng(0)
rng.shuffle(eval_theorems)
cutoff = 4096
train_theorems = [{'prompt': entry[1], 'target': entry[2]} for entry in eval_theorems[cutoff:]]
eval_theorems = [{'prompt': entry[1], 'target': entry[2]} for entry in eval_theorems[:cutoff]]

In [18]:
with open(os.path.join(Save_DIR, 'eval.json'), 'w') as f:
    json.dump(eval_theorems, f)

## mathlib + Conjecture

In [19]:
train_ds = conjecture_ds + train_theorems
rng = np.random.default_rng(0)
rng.shuffle(train_ds)
print(len(train_ds))

106921


In [20]:
with open(os.path.join(Save_DIR, 'mathlib_conjecture.json'), 'w') as f:
    json.dump(train_ds, f)